In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import traceback
import re
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import queue
import csv

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s', stream=sys.stdout)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device 
gpio_pins_per_device = {
    'Port 1': {"LeftPoke": 17, "RightPoke": 27, "Pellet": 22},
    'Port 2': {"LeftPoke": 10, "RightPoke": 9, "Pellet": 11},
    'Port 3': {"LeftPoke": 0, "RightPoke": 5, "Pellet": 6},
    'Port 4': {"LeftPoke": 13, "RightPoke": 19, "Pellet": 26},
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Global event to signal threads to stop
stop_event = threading.Event()

# Define the column headers for CSV
column_headers = [
    "Timestamp", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.geometry("800x480")
        self.root.title("HPFED DATA MONITOR V.01")

        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.save_path = ""

        # Data storage during the session
        self.data_to_save = {}
        self.setup_gui()

    def setup_gui(self):
        # GUI setup
        tk.Label(self.root, text="Your Name:", font=("Helvetica", 8, "bold")).grid(column=0, row=0, sticky=tk.E, padx=2, pady=2)
        self.experimenter_entry = ttk.Entry(self.root, textvariable=self.experimenter_name)
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W, padx=2, pady=2)
    
        tk.Label(self.root, text="Experiment Name:", font=("Helvetica", 8, "bold")).grid(column=2, row=0, sticky=tk.E, padx=2, pady=2)
        self.experiment_entry = ttk.Entry(self.root, textvariable=self.experiment_name)
        self.experiment_entry.grid(column=3, row=0, sticky=tk.W, padx=2, pady=2)

        # Start and Stop buttons
        self.start_button = tk.Button(self.root, text="START", font=("Helvetica", 10, "bold"), bg="green", fg="white", command=self.start_experiment)
        self.start_button.grid(column=1, row=3, padx=2, pady=5)
        self.stop_button = tk.Button(self.root, text="STOP (SAVE & QUIT)", font=("Helvetica", 10, "bold"), bg="red", fg="white", command=self.stop_experiment)
        self.stop_button.grid(column=2, row=3, padx=2, pady=5)

        # Browse for Data Folder
        self.browse_button = tk.Button(self.root, text="Browse Data Folder", font=("Helvetica", 10, "bold"), command=self.browse_folder, bg="gold", fg="blue")
        self.browse_button.grid(column=0, row=3, padx=2, pady=5)

        self.canvas = tk.Canvas(self.root, width=100, height=100)
        self.canvas.grid(column=1, row=4, columnspan=2, padx=2, pady=2)
        self.recording_circle = None

        self.setup_ports()

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")

    def setup_ports(self):
        port_names = ['Port 1', 'Port 2', 'Port 3', 'Port 4']
        for idx, port_name in enumerate(port_names):
            frame = ttk.LabelFrame(self.root, text=port_name)
            frame.grid(column=idx, row=6, padx=2, pady=2, sticky=(tk.N, tk.S, tk.E, tk.W))
            status_label = ttk.Label(frame, text="Not Ready", font=("Helvetica", 8, "italic"), foreground="red")
            status_label.grid(column=0, row=0, sticky=tk.W)
            text_widget = tk.Text(frame, width=20, height=10)
            text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))
            self.port_widgets[port_name] = {'status_label': status_label, 'text_widget': text_widget}
            self.port_queues[port_name] = queue.Queue()

    def start_experiment(self):
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experimenter_name = self.experimenter_name.get().lower().strip()
        experiment_name = self.experiment_name.get().lower().strip()
        
        if not self.save_path:
            messagebox.showerror("Error", "Please select a folder to save data.")
            return
        
        experimenter_folder = os.path.join(self.save_path, experimenter_name)
        experiment_folder = os.path.join(experimenter_folder, f"{experiment_name}_{current_time}")
        os.makedirs(experiment_folder, exist_ok=True)
        self.experiment_folder = experiment_folder
        self.data_to_save = {port_identifier: [] for port_identifier in self.port_widgets.keys()}

        self.threads = []
        for port_identifier, q in self.port_queues.items():
            self.data_to_save[port_identifier] = []
            t = threading.Thread(target=self.read_from_fed, args=(port_identifier, q))
            t.daemon = True
            t.start()
            self.threads.append(t)

    def read_from_fed(self, port_identifier, queue):
        # Simulate reading from port and pushing to the queue
        while not stop_event.is_set():
            # Simulated data fetch; replace with actual port reading code
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
            data_row = [timestamp] + [f"data_{i}" for i in range(len(column_headers)-1)]
            queue.put(data_row)
            time.sleep(1)  # Simulate data arrival time

    def stop_experiment(self):
        stop_event.set()
        for t in self.threads:
            t.join()
        GPIO.cleanup()
        self.save_all_data()
        self.root.quit()
        self.root.destroy()

    def save_all_data(self):
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        for port_identifier, data_rows in self.data_to_save.items():
            # File with date and time in both locations
            filename_user = f"{self.experiment_folder}/{port_identifier}_{current_time}.csv"
            filename_direct = f"{self.save_path}/{port_identifier}_{current_time}.csv"
            for filename in [filename_user, filename_direct]:
                with open(filename, mode='w', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(column_headers)
                    writer.writerows(data_rows)

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()
